# 제너레이터

## 30 리스트를 반환하기 보다는 제너레이터를 사용하라.
- 또하나의 차이점은 for문의 index_words는 모든 결과값을 저장한다. 
- 제너레이터 버전으로 만들면 사용하는 메모리 크기를 어느정도 제한할 수 있다. -> 입력 길이가 아무리 길어도 쉽게 처리가 가능하다. 


In [3]:
# 일반적인 for문 
def index_words(text):
    result =[]
    if text:
        result.append(0)
    for ind,val in enumerate(text):
        if val == " ":
            result.append(ind+1)
    
    return result 

address = "컴퓨터(영어: Computer, 문화어: 콤퓨터, 순화어: 전산기)는 진공관"
result = index_words(address)
print(result[:10])

# 위 일반적인 for문은 핵심을 알아보기 어려움 - 가독성이 떨어짐 
# index +1의 중요성을 희석해 버림 
# 제너레이터 방법
def index_words_iter(text):
    if text:
        yield 0 
    for ind,val in enumerate(text):
        if val == " ":
            yield ind+1

# 가독성이 늘어 났다 -> 반환하는 리스트와 상호작용하는 코드가 사라짐 
# 이 함수가 호출되면 제너레이터 함수가 실제로 실행되지 않고 이터레이터를 반환한다. 
# 이터레이터가 next내장 함수를 호출할 때마다 이터레이터는 제너레이터 함수를 다음 yield까지 진행 
# 제너레이터가 yield에 전강하는 값은 이터레이터에 의해 호출하는 쪽에 반환한다.!!!! 
            
it = index_words_iter(address)
print(next(it))
print(next(it))

# 제너레이터의 출력이 이터레이터를 반환하기 때문에 손쉽게 리스트로 바꾸어 확인이 가능하다. 
result = list(index_words_iter(address))
print(result[:10])

[0, 8, 18, 23, 28, 33, 39]
0
8
[0, 8, 18, 23, 28, 33, 39]


- 또 하나의 차이점은 for문의 index_words는 모든 결과를 저장한다. 
- 제너레이터 버전으로 만들면 사용하는 메모리 크기를 어느정도 제한 가능하다. -> 입력 길이가 아무리 길어도 제일 긴 줄 정도


In [4]:
import itertools
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset +=1
            if letter == " ":
                yield offset

# 이 함수의 작업 메모리는 입력 중 가장 긴 줄의 길이로 제한 된다. 
# 제너레이터를 정의할 때 한 가지 주의함 점이 있다.
#제너레이터가 반환하는 이터레이터에 상태가 있기에 호출하는 쪽에서 재사용이 불가능 하다
with open("address.txt","r",encoding="utf-8") as f:
    it = index_file(f)
    results = itertools.islice(it,0,10) # 36. 이터레이터나 제너레이터를 다룰 때는 itertools를 사용하라.
    print(list(results))


[0, 8, 18, 23, 28, 38]


제너레이터를 정의할 대 한 가지 알아둬야 할 사항은 제너레이터가 반환하는 이터레이터에 상태가 있기 때문에 
호출하는 쪽에서 재상용이 불가능하다.

## 31. 인자에 대해 이터레션할 때는 방어적이 돼라.
- 객체가 원소로 들어가 있는 리스트를 함수가 파라미터로 받았을 때, 이 리스트를 여러번 이터레이션 하는 것이 중요할 때가 종종 있다.
- 전체 여행자수를 계산하기 위해 입력 전체의 합계를 내고 이 합계를 이용해서 각 도시의 방문자 수를 나누는 정규화 함수가 필요하다. 

In [5]:
def normalize(numbers):
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100*value /total
        result.append(percent)
    return result

visits = [15,35,80]
percentages = normalize(visits)
print(percentages)

assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [6]:
# 코드의 확장성을 높이고, 객체 수를 증가 시킨다면, 메모리가 중요하다.(훨씬 더 많은 메모리 필요)
# 그러므로 파일을 읽는 제너레이터를 구현 한다. 
def read_visits(data_path):
    with open(data_path) as f:
        for line in f: # txt 파일이지만, 'r' 옵션도 주지 않고, f.readline등의 함수도 사용하지 않는다? 
            yield int(line) # readlines랑 같은 역활?

it = read_visits("my_numbers.txt")
percentages = normalize(it)
print(percentages)


[]


빈 리스트가 출력되는 이유는 제너레이터 타입이 input 으로 들어가기 때문에   
sum()함수에서 제너레이터 함수를 전부 사용하고 그 밑에 for문은 빈 리스트를 실행함    
이미 stopIteration이 발생했지만 파이썬 표준 라이브러리에 있는 많은 함수가 일반적인 연산 도중에 stopIteration 예외가   
던져지는 것을 가정한다.

In [7]:
it = read_visits("my_numbers.txt")
print(list(it))
print(list(it))

[15, 35, 80]
[]


- 이런 함수들은 출력이 없는 이터레이터와 이미 소진돼버린 이터레이터를 구분할 수 없다. 
- 이 문제를 해결하기 위해 입력 이터레이터를 명시적으로 소진 시키고, 이터레이터의 전체 내용을 리스트에 넣을 수 있다. 
- 리스트에 넣은 후 원하는 만큼 반복해서 이터레이션을 수행 할 수 있다. 

In [8]:
# 입력 이터레이터를 방어적으로 복사하도록 만든 코드다. 
def normalize_copy(numbers):
    numbers_copy = list(numbers)
    total = sum(numbers_copy)
    result = [] 
    for value in numbers_copy:
        percent = 100 * value / total
        result.append(percent)
    return result

it = read_visits("my_numbers.txt")
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

# 단 이 방법은 메모리 문제가 그대로 있다. 
# copy하면서 리스트 길이 만큼 메모리를 잡아먹기 때문이다. 
# 제너레이터를 쓰는 이유가 반감이 되어 버린다.


[11.538461538461538, 26.923076923076923, 61.53846153846154]


- 위 방법(리스트에 복사 하는 방법은) 이전에 문제가 발생했던 메모리 문제를 해결하지 못한다. (규모 확장성)
- 이 문제를 해결하는 다른 방법은 호출될 때마다 새로운 이터레이터를 반홚나는 함수를 받는 것이다. 

In [9]:
# 이 문제를 해결하는 다른 방법으로는 호출 될 때마다 새로 이터레이터를 반환하는 함수를 받는 것이다.
def normalize_func(get_iter):
    total = sum(get_iter()) # 새 이터레이터? 
    result = [] 

    for value in get_iter(): 
        percent = 100 *value / total
        result.append(percent)
    
    return result 
path = "my_numbers.txt"
percentages = normalize_func(lambda : read_visits(path))
print(percentages)
assert sum(percentages) == 100.0
# 잘 작동하지만, 람다 함수를 넘기는 것은 보기에 좋치 않다
# 같은 결과를 달성하는 더 나은 방법은 이터레이터 프로토콜을 구현한 새로운 컨테이너 클래스를 제공하는 것이다. 


[11.538461538461538, 26.923076923076923, 61.53846153846154]



# 이터레이터 프로토콜
- 이터레이터 프로토콜은 파이썬의 For루프나 그와 관련된 식들이 컨테이너타입의 내용을 방문할 때 사용하는 절차이다. 
- 파이썬에서 for x in foo 구문을 사용하면 실제로는 iter(foo)를 호출한다.
- iter 내장함수는 foo.\_\_iter\_\_ 라는 특별 메소드를 호출한다. 
- \_\_iter\_\_메서드는 반드시 이터레이터 객체를 반환해야 한다. for 루프는 반환받은 이터레이터 객체가 데이터를 소진(StopIteration 예외)를 받을 때까지 반복적으로 이터레이터 객체에 대해 Next 내장함수를 호출한다. 


In [10]:
class ReadVisit: # 이터러블 컨테이너 타입 
    def __init__(self,data_path) -> None:
        self.data_path = data_path

    def __iter__(self):
        with open(self.data_path) as f:
            for line in f:
                yield int(line)


- \_\_iter\_\_ 메서드를 제너레이터로 구현하기만 하면 된다. 
- 메모리 이슈는 발생 여부가 낮아지지만, 이터레이터를 여러번 이터레이션 해야한다. 

In [11]:
visits = ReadVisit(path)
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


normalize 함수 안의 sum메서드가 ReadVisit.\_\_iter\_\_ 를 호출해서 새로운 이터레이터 객체를 할당하고   
for 루프 또한 ReadVisit.\_\_iter\_\_ 호출해서 두 번쨰 이터레이터 객체를 만든다.   

이 두 이터레이터는 서로 독립적으로 진행되고 소진된다.   

이 접근 방법의 유일한 단점은 입력데이터를 여러 번 읽는다는 것이다. 

- 이터레이터가 iter 내장 함수에 전달되는 경우에는 전달받은 이터레이터가 그대로 반환된다.
- 컨테이너 타입이 iter에 전달되면 매번 새로운 이터레이터 객체가 반환된다. 
- 반복적으로 이터레이션 할 수 없는 인자인 경우에는 TypeError를 발생시켜서 인자를 거부한다. 

In [12]:
def normalize_defensive(numbers):
    if iter(numbers) is numbers: # 이터레이터 객체가 들어오면 반복적으로 이터레이션 할 수 없이, 한번 사용하고 소진된다. 
        raise TypeError
    # List는 문제가 되지 않는데, 리스트 또한 이터레이터 프로토콜을 따르는 이터러블 컨테이너기 떄문이다. 
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result 

pers = normalize_defensive(visits) # 컨테이너 반환 
print(pers)

per_err = normalize_defensive(read_visits(path)) # 이터레이터 반환
print(per_err)


[11.538461538461538, 26.923076923076923, 61.53846153846154]


TypeError: 

In [15]:
from collections.abc import Iterator 

def normalize_defensive2(numbers):
    if isinstance(numbers,Iterator):
        raise TypeError
    total = sum(numbers)
    result = [] 
    for value in numbers:
        percent = 100 *value / total
        result.append(percent)
    return result


pers = normalize_defensive2(visits) # 컨테이너 입력 
print(pers)

visit = [15,35,80]
per_err2 = normalize_defensive2(visit) #리스트 입력
print(per_err2)

[11.538461538461538, 26.923076923076923, 61.53846153846154]
[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [16]:
per_err = normalize_defensive2(read_visits(path)) # 이터레이터 반환
print(per_err)

TypeError: 

In [ ]:
per_err4 = normalize_defensive2(iter(visit))
print(per_err4)

TypeError: 

# 리스트 컴프리헨션보다 제너레이터 식을 사용하라. 
- 각 줄에 들어 있는 문자 수를 반환한다고 하자. 
- 이를 리스트 컴프리헨션으로 하려면 파일 각 줄의 길이를 메모리에 저장해야한다. 
- 파일이 아주 크거나 절대로 끝나지 않는 네트워크 소켓이라면 리스트 컴프리헨셔을 사용하는 것은 문제가 될 수 있다. 

In [ ]:
# 입력이 작은 경우 리스트 컴프리헨션을 사용한다. 
value = [len(x) for x in open("my_file.txt")]
print(value)



# 제너레이터 식 
it = (len(x) for x in open("my_file.txt"))
print(it) # 제너레이터 식은 출력 시퀀스 전체가 실체화 되지 않는다
print(next(it))
print(next(it))

# 제너레이터의 강력한 특징은 두 제너레이터 식을 합성할 수 있다는 점이다. 
# 다음 코드에서는 앞에서 본 제너레이터 식이 반환한 이터레이터를 다른 제너레이터 식의 입력으로 사용한다. 

Root = ((x,x**0.5) for x in it)
print(next(Root))



[100, 57, 15, 1, 12, 75, 5, 86, 89, 10]
<generator object <genexpr> at 0x13021f580>
100
57
(15, 3.872983346207417)


제너레이터 식의 또 다른 강력한 특징은 두 제너레이터 식을 합성할 수 있다는 점이다.    


제너레이터 식은 이터레이터를 전진시킬 때마다 내부의 이터레이도 전진되면서 도미노처럼 연쇄적으로 루프가 실행돼 조건식을 평가하고,   
입력과 출력을 서로 주고 받는다. 이 모든 과정이 가능한 **메모리를 효율적**으로 사용하면서 이뤄진다. 


## 33. yield from을 사용해 여러 제너레이터를 합성하라. 
- 제너레이터가 아주 유용하기 때문에 다양한 곳에 제너레이터가 쓰이고 있다. 
- 이로 인해 제너레이터를 여러 단계에 걸쳐 한 줄기로 연결한 것처럼 보이는 프로그램도 많다.



In [19]:
def move(period,speed):
    for _ in range(period):
        yield speed 

def pause(delay):
    for _ in range(delay):
        yield 0 

# 두 제네레이터 함수를 합성해서 변위 시퀀스를 하나만 만들어야 한다. 
# 애니메이션의 각 단계마다 제너레이터를 호출해서 차례로 이터레이션하고 
# 각 이터레이션에서 나오는 변위를 순서대로 내보내는 방식으로 다음과 같이 시퀀스를 만든다. 
        
def animate():
    for delta in move(4,5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2,3.0):
        yield delta 

def render(delta):
    print(f"Delta : {delta:.1f}")

def run(func):
    for delta in func():
        render(delta)
        
run(animate)


Delta : 5.0
Delta : 5.0
Delta : 5.0
Delta : 5.0
Delta : 0.0
Delta : 0.0
Delta : 0.0
Delta : 3.0
Delta : 3.0


- 이 코드의 문제점은 animate가 너무 반복적이라는 것이다. 
for문과 yield 식이 반복되면서 잡음이 늘고 가독성이 줄어든다.   
이 예제는 제너레이터를 겨우 세개만 내포 시켰는데 벌써 코드가 명확하지 못하다. 

열 단계가 넘어가는 복잡한 애니메이션을 표현하는 코드는 따라가기 훨씬 더 어려울 것이다. 


- 이 문제의 해법은 yield from 식을 사용하는 것이다. 이는 고급 제너레이터 기능으로, 
- 제어를 부모 제너레이터에게 전달하기 전에 내포된 제너레이터가 모든 값을 내보낸다. 


In [20]:
def animate_composed():
    yield from move(4,5.0)
    yield from pause(3)
    yield from move(2,3.0)

run(animate_composed)

Delta : 5.0
Delta : 5.0
Delta : 5.0
Delta : 5.0
Delta : 0.0
Delta : 0.0
Delta : 0.0
Delta : 3.0
Delta : 3.0


- 좀 더 명확하고 더 직관적이다. 

yield from은 근본적으로 파이썬 인터프리터가 대신 for 루프를 내포시키고 yield 식을 처리하도록 만든다.    
이로 인해 성능도 더 좋아진다. 다음 코드에서는 timeit내장 모듈을 통해 마이크로 벤치마크를 실행함으로써 성능이 개선되는지 살펴보자.

In [22]:
import timeit 

def child():
    for i in range(1_000_000):
        yield i
    
def slow():
    for i in child():
        yield i 
def fast():
    yield from child()

baseline = timeit.timeit(
    stmt = "for _ in slow():pass",
    globals=globals(),
    number=50
)

print(f"수동 내포 : {baseline:.2f}s")

comparison = timeit.timeit(
    stmt="for _ in fast():pass",
    globals = globals(),
    number=50
)

print(f"합성 사용 : {comparison:.2f}s")

reduction = -(comparison-baseline) / baseline
print(f"{reduction:.1%} 시간이 적게 듦")


수동 내포 : 3.37s
합성 사용 : 2.96s
12.2% 시간이 적게 듦


## send로 제너레이터에 데이터를 주입하지 말라.
- yield를 사용하면 제너레이터 함수가 간단하게 이터레이션이 가능한 출력 값을 만들어 낼 수 있다. 
- 하지만 이렇게 만들어 내는 데이터 채널은 단방향이다. 
- 제너레이터가 데이터를 내보내면서 다른 데이터를 받아들일 때 직접 쓸 수 있는 방법이 없는 것처럼 보인다. 

- 하지만 양방향 통신이 있다면 많은 경우에 도움이 될 것이다. 

In [23]:
# 소프트웨어 라디오를 사용해 신호를 내보낸다고 하자. 
# 다음 코드는 주어진 간격과 진폭에 따른 사인파 값을 생성한다. 

import math 

def wave(amplitude,steps):
    step_size = 2*math.pi/steps # 2라이안 / 단계 수 
    for step in range(steps):
        radians = step*step_size
        fraction = math.sin(radians)
        output = amplitude * fraction
        yield output 


def transmit(output):
    if output is None:
        print(f"출력 : None")
    else:
        print(f"출력 : {output:>5.1f}")

def run(it):
    for output in it:
        transmit(output)


run(wave(3.0,8))

출력 :   0.0
출력 :   2.1
출력 :   3.0
출력 :   2.1
출력 :   0.0
출력 :  -2.1
출력 :  -3.0
출력 :  -2.1


In [25]:
# 라디오 신호를 방송하기 위해 필요한 다른 신호를 사용해 진폭을 지속적으로 변경해야 한다면,
# 이 코드는 쓸모가 없다. 우리에게는 제너레이터를 이터레이션할 때마다 진폭을 변조할 수 있는 방법이 필요하다. 

# 제네레이터는 send 메서드를 지원한다. 이 메서드는 yield 식을 양방향 채널로 격상시켜준다. 
# send 메서드를 사용하면 입력을 제너레이터에 스트리밍하는 동시에 출력을 내보 낼 수 있다. 
# 일반적으로 제너레이터를 이터레이션 할 때 yield식이 반환하는 값은 None이다. 

def my_generator():
    received = yield 1
    print(f"받은 값 = {received}")

it = iter(my_generator())
output = next(it)
print(f"출력값 = {output}")

try:
    next(it) # 종료될 때까지 제너레이터를 실행한다. 
except StopIteration:
    pass 


출력값 = 1
받은 값 = None


- for , next 내장 함수로 제너레이터를 이터레이션 하지 않고 send메서드를 호출하면, 제너레이터가 재개(resume)될 때,
- yield 가 send에 전달된 파라미터 값을 반환한다. 
- 하지만 방금 시작한 제너레이터는 아직 yield 식에 도달하지 못했기 때문에 최초로 send를 호출할 때 인자로 전달할 수 있는 유일한 값은 None뿐이다. 

In [27]:
it = iter(my_generator())
output = it.send(None) # 첫 번째 제너레이터 출력을 얻는다.
# None이 아니면 TypeError: can't send non-None value to a just-started generator 발생 

print(f"출력값 = {output}")

try:
    it.send("안녕!")
except StopIteration:
    pass 

TypeError: can't send non-None value to a just-started generator